In [ ]:
import pandas as pd
import numpy as np
from models.models import TextClassificationNet
from models.utils.data_loader import TextDataloader
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import tensorflow as tf

In [ ]:

def predict(data, model_name, weights_path):
    ''' 
    Predict function for text classification

    ----------------
    INPUTS:
    data: Pandas.DataFrame - text column named 'text', label column named 'class' with possible values [0, 1, 2] (0 - normal language, 1 - hate speech, 2 - offensive language)
    model_name: String - possible values ['bert', 'csebert', 'xlmr']
    weights_path: String - path to weights file

    ----------------
    OUTPUTS:
    preds: List - list of softmax predictions for each input text
    '''
    preds = []

    text_dataloader = TextDataloader(seq_length=SEQ_LENGTH, batch_size=32, language_model=model_name)
    tcn = TextClassificationNet(seq_length=SEQ_LENGTH, language_model=model_name)
    model = tcn.get_model()
    model.load_weights(weights_path)

    bs = 64
    with tqdm(total=df.shape[0] // bs) as pbar:
        for i,rows in df.groupby(np.arange(len(df))//bs):
            pbar.update(1)
            tokens, masks = zip(*map(text_dataloader.tokenize, rows['text'].to_numpy()))
            if len(masks) > 1 and len(tokens) > 1:
                tokens, masks = tf.squeeze(tf.stack(tokens)), tf.squeeze(tf.stack(masks))

            predicted = model.predict([tokens, masks])
            preds += list(predicted)

    return preds

def evaluate(labels, predictions, name='generic_network'):
    print('=================== ' + name + ' ===================\n')
    print(confusion_matrix(labels, predictions))
    print(classification_report(labels, predictions, digits=3))
    print('\n\n')

def ensemble_voting(model_preds):
    ''' 
    Combine predictions for ensemble voting

    ----------------
    INPUTS:
    model_preds: List - 3d list of softmax predictions for each model contributing to voting

    ----------------
    OUTPUTS:
    preds: List - 1d list of predictions
    '''

    return (np.sum(model_preds, axis=0) / len(model_preds)).argmax(axis=-1)


In [ ]:
# TODO create Pandas DataFrame with text column named 'text' and labels column names 'class'

df = None

In [ ]:
mbert_predictions = predict(df, model_name='bert', weights_path='/path/to/weights.h5')
csebert_predictions = predict(df, model_name='csebert', weights_path='/path/to/weights.h5')
xlmr_predictions = predict(df, model_name='xlmr', weights_path='/path/to/weights.h5')

labels = df['class']

ensemble_preds = [mbert_predictions, csebert_predictions, xlmr_predictions]
evaluate(labels, ensemble_preds, 'Ensemble')

mbert_preds = mbert_predictions.argmax(axis=-1)
evaluate(labels, mbert_preds, 'mBERT')

csebert_preds = csebert_predictions.argmax(axis=-1)
evaluate(labels, csebert_preds, 'CroSloEngualBERT')

xlmr_preds = xlmr_predictions.argmax(axis=-1)
evaluate(labels, xlmr_preds, 'XLMr')
